# EFDTモデルためのデータ処理

目的：各時点でのBollinger Bands, RSI, RCI, Momentumを計算する  
開発環境： python 3.8.10/ JupyterLab 2.3.2/ System Information: System: Linux (179-Ubuntu SMP)

## 必要なライブラリをインポート

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Pandas version: 1.5.2   
Numpy version: 1.22.2   
Scikit-learn version: 1.2.0

In [49]:
# USDJPY通貨ペアをDataFrameに読み込む
USDJPY = pd.read_csv('USDJPY.csv',sep = ',')

# 'Times'列を日時形式に変換する
USDJPY['Times'] = pd.to_datetime(USDJPY['Times'], format='%Y%m%d %H%M%S',errors='coerce')

# 'Times'列を基準に重複する行を削除する
USDJPY = USDJPY.drop_duplicates(subset=['Times'])

display(USDJPY)

,Times,Final Price
0,2018-01-01 17:01:00,112.616
1,2018-01-01 17:02:00,112.634
2,2018-01-01 17:03:00,112.620
3,2018-01-01 17:04:00,112.613
4,2018-01-01 17:05:00,112.620
...,...,...
2139460,2023-11-17 16:54:00,149.638
2139461,2023-11-17 16:55:00,149.633
2139462,2023-11-17 16:56:00,149.622
2139463,2023-11-17 16:57:00,149.619


In [50]:
# 欠損値を補間する
USDJPY.set_index('Times', inplace=True)
USDJPY = USDJPY.resample('1T').interpolate().reset_index()

# 'Final Price'列を正規化し、正規化された値を新しい列に追加する
scaler = MinMaxScaler()
USDJPY['Final Price Normalized'] = scaler.fit_transform(USDJPY[['Final Price']])

display(USDJPY)

,Times,Final Price,Final Price Normalized
0,2018-01-01 17:01:00,112.616,0.224973
1,2018-01-01 17:02:00,112.634,0.225328
2,2018-01-01 17:03:00,112.620,0.225052
3,2018-01-01 17:04:00,112.613,0.224914
4,2018-01-01 17:05:00,112.620,0.225052
...,...,...,...
3090233,2023-11-17 16:54:00,149.638,0.954816
3090234,2023-11-17 16:55:00,149.633,0.954718
3090235,2023-11-17 16:56:00,149.622,0.954501
3090236,2023-11-17 16:57:00,149.619,0.954442


Compute technical indicators, including bollinger bands, rsi, rci, momentum

In [51]:
# ボリンジャーバンド計算
def calculate_bollinger_bands(data):
    
    # Use the last 60 minutes to calculate Bollinger Bands
    window = 60
    
    # Check if there are enough data points
    if len(data) < window:
        return np.nan, np.nan, np.nan
    else:
        # Calculate rolling mean, upper band, and lower band
        rolling_mean = data['Final Price'].rolling(window=window).mean()
        upper_band = rolling_mean + 2 * data['Final Price'].rolling(window=window).std()
        lower_band = rolling_mean - 2 * data['Final Price'].rolling(window=window).std()

        return upper_band.iloc[-1], lower_band.iloc[-1], rolling_mean.iloc[-1]

# RSI計算
def calculate_rsi(data):
    
    # Use the last 60 minutes to calculate RSI
    window = 60
    
    # Check if there are enough data points
    if len(data) < window:
        return np.nan
    else:
        # Calculate differences, gains, and losses
        diff = data['Final Price'].diff(1).dropna().tail(window)
        gain = diff.where(diff > 0, 0)
        loss = -diff.where(diff < 0, 0)
        
        # Calculate average gain and average loss
        avg_gain = gain.mean()
        avg_loss = loss.mean()
        
        # Calculate relative strength (rs)
        if avg_loss == 0:
            rs = np.inf  # Set to infinity to avoid division by zero
        else:
            rs = avg_gain / avg_loss

        rsi = 100 - (100 / (1 + rs))
        return rsi

# RCI計算
def calculate_rci(data):
    
    # Use the last 60 minutes to calculate RSI
    window = 60
    
    # Check if there are enough data points
    if len(data) < window:
        return np.nan
    else:
        rci = data['Final Price'].pct_change().tail(window).sum()
        return rci

# モメンタム計算
def calculate_momentum(data, n=59):
    
    # Use the last 60 minutes to calculate RSI
    window = 60
    
    # Check if there are enough data points
    if len(data) < window:
        return np.nan
    else:
        return data['Final Price'].diff(n).iloc[-1]


In [52]:
# 各時点でのボリンジャーバンド、RSI、RCI、モメンタムを計算する
for index, row in tqdm(USDJPY.iterrows()):
    upper_band, lower_band, rolling_mean = calculate_bollinger_bands(USDJPY.iloc[index-60:index])
    rsi_value = calculate_rsi(USDJPY.iloc[index-60:index])
    rci_value = calculate_rci(USDJPY.iloc[index-60:index])
    momentum = calculate_momentum(USDJPY.iloc[index-60:index])
    

    USDJPY.loc[index, 'Upper Band'] = upper_band
    USDJPY.loc[index, 'Lower Band'] = lower_band
    USDJPY.loc[index, 'Rolling Mean'] = rolling_mean
    USDJPY.loc[index, 'RSI'] = rsi_value
    USDJPY.loc[index, 'RCI'] = rci_value
    USDJPY.loc[index, 'Momentum'] = momentum

print(USDJPY)

0it [00:00, ?it/s]

                      Times  Final Price  Final Price Normalized  Upper Band  \
0       2018-01-01 17:01:00      112.616                0.224973         NaN   
1       2018-01-01 17:02:00      112.634                0.225328         NaN   
2       2018-01-01 17:03:00      112.620                0.225052         NaN   
3       2018-01-01 17:04:00      112.613                0.224914         NaN   
4       2018-01-01 17:05:00      112.620                0.225052         NaN   
...                     ...          ...                     ...         ...   
3090233 2023-11-17 16:54:00      149.638                0.954816  149.723578   
3090234 2023-11-17 16:55:00      149.633                0.954718  149.724052   
3090235 2023-11-17 16:56:00      149.622                0.954501  149.724646   
3090236 2023-11-17 16:57:00      149.619                0.954442  149.725883   
3090237 2023-11-17 16:58:00      149.587                0.953811  149.726948   

         Lower Band  Rolling Mean      

In [53]:
# 'Nan'の部分を切り捨てる
USDJPY.set_index('Times', inplace=True)

start_date = '2018-01-01 18:09:00'
end_date = '2023-11-17 16:54:00'

USDJPY = USDJPY.loc[start_date:end_date]

USDJPY.reset_index(inplace=True)

display(USDJPY)

,Times,Final Price,Final Price Normalized,Upper Band,Lower Band,Rolling Mean,RSI,RCI,Momentum
0,2018-01-01 18:09:00,112.650,0.225644,112.657555,112.561912,112.609733,56.844548,0.000524,0.059
1,2018-01-01 18:10:00,112.635,0.225348,112.659427,112.561673,112.610550,53.990610,0.000302,0.034
2,2018-01-01 18:11:00,112.646,0.225565,112.660132,112.561601,112.610867,51.605505,0.000125,0.014
3,2018-01-01 18:12:00,112.645,0.225545,112.661314,112.561253,112.611283,52.796421,0.000222,0.025
4,2018-01-01 18:13:00,112.642,0.225486,112.662409,112.560958,112.611683,52.466368,0.000196,0.022
...,...,...,...,...,...,...,...,...,...
3090161,2023-11-17 16:50:00,149.644,0.954934,149.721929,149.635588,149.678758,48.203593,-0.000080,-0.012
3090162,2023-11-17 16:51:00,149.642,0.954895,149.722299,149.634751,149.678525,47.916667,-0.000093,-0.014
3090163,2023-11-17 16:52:00,149.641,0.954875,149.722731,149.633786,149.678258,47.477745,-0.000113,-0.017
3090164,2023-11-17 16:53:00,149.642,0.954895,149.723195,149.632721,149.677958,47.619048,-0.000107,-0.016


In [54]:
# 処理済みのデータを保存する
USDJPY.to_csv('USDJPY_ti.csv', index=False)

In [55]:
USDJPY_ti = pd.read_csv('USDJPY_ti.csv',sep = ',')

display (USDJPY_ti)

,Times,Final Price,Final Price Normalized,Upper Band,Lower Band,Rolling Mean,RSI,RCI,Momentum
0,2018-01-01 18:09:00,112.650,0.225644,112.657555,112.561912,112.609733,56.844548,0.000524,0.059
1,2018-01-01 18:10:00,112.635,0.225348,112.659427,112.561673,112.610550,53.990610,0.000302,0.034
2,2018-01-01 18:11:00,112.646,0.225565,112.660132,112.561601,112.610867,51.605505,0.000125,0.014
3,2018-01-01 18:12:00,112.645,0.225545,112.661314,112.561253,112.611283,52.796421,0.000222,0.025
4,2018-01-01 18:13:00,112.642,0.225486,112.662409,112.560958,112.611683,52.466368,0.000196,0.022
...,...,...,...,...,...,...,...,...,...
3090161,2023-11-17 16:50:00,149.644,0.954934,149.721929,149.635588,149.678758,48.203593,-0.000080,-0.012
3090162,2023-11-17 16:51:00,149.642,0.954895,149.722299,149.634751,149.678525,47.916667,-0.000093,-0.014
3090163,2023-11-17 16:52:00,149.641,0.954875,149.722731,149.633786,149.678258,47.477745,-0.000113,-0.017
3090164,2023-11-17 16:53:00,149.642,0.954895,149.723195,149.632721,149.677958,47.619048,-0.000107,-0.016
